<a href="https://colab.research.google.com/github/mitesh98/Quora_Question_Pair_Similarity/blob/master/Quora_Question_Similarity_2_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**1.2.1 : EDA: Advanced Feature Extraction.**

In [ ]:
!pip install fuzzywuzzy

In [ ]:
!pip install distance

     |████████████████████████████████| 184kB 2.9MB/s 
  Created wheel for distance: filename=Distance-0.1.3-cp36-none-any.whl size=16261 sha256=8213ab6243ec99d7125905539929a2741f9a582860da8d4fd5e3360a9b55d0d5
  Stored in directory: /root/.cache/pip/wheels/d5/aa/e1/dbba9e7b6d397d645d0f12db1c66dbae9c5442b39b001db18e
Successfully built distance


In [ ]:
import warnings
warnings.filterwarnings("ignore")
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from subprocess import check_output
%matplotlib inline
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls
import os
import gc

import re
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from bs4 import BeautifulSoup
import re
from nltk.corpus import stopwords
# This package is used for finding longest common subsequence between two strings
import distance
# you can write your own dp code for this
from nltk.stem import PorterStemmer
from bs4 import BeautifulSoup
from sklearn.manifold import TSNE
# Import the Required lib packages for WORD-Cloud generation
# https://stackoverflow.com/questions/45625434/how-to-install-wordcloud-in-python3-6
from wordcloud import WordCloud, STOPWORDS
from os import path
from PIL import Image
from fuzzywuzzy import fuzz

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
!cp '/content/drive/My Drive/Colab Notebooks/train2.csv' 'train2.csv'

In [ ]:
data=pd.read_csv('train2.csv')
data.head()

,id,qid1,qid2,question1,question2,is_duplicate,freq_qid1,freq_qid2,q1len,q2len,q1_n_words,q2_n_words,word_Common,word_Total,word_Share,freq_q1+q2,freq_q1-q2
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0,1,1,66,57,14,12,10.0,23.0,0.434783,2,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0,4,1,51,88,8,13,4.0,20.0,0.200000,5,3
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0,1,1,73,59,14,10,4.0,24.0,0.166667,2,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0,1,1,50,65,11,9,0.0,19.0,0.000000,2,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0,3,1,76,39,13,7,2.0,20.0,0.100000,4,2


**3.4 Preprocessing of Text**

* Preprocessing:

* Removing html tags

* Removing Punctuations

* Performing stemming

* Removing Stopwords

* Expanding contractions etc.

In [ ]:
import nltk
nltk.download('stopwords')
Stopwords=stopwords.words('english')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
def Preprocessing(x):
  x=x.replace(",000,000",'m').replace(",000",'k').replace("′", "'").replace("’", "'")\
               .replace("won't", "will not").replace("cannot", "can not").replace("can't", "can not")\
                           .replace("n't", " not").replace("what's", "what is").replace("it's", "it is")\
                           .replace("'ve", " have").replace("i'm", "i am").replace("'re", " are")\
                           .replace("he's", "he is").replace("she's", "she is").replace("'s", " own")\
                           .replace("%", " percent ").replace("₹", " rupee ").replace("$", " dollar ")\
                           .replace("€", " euro ").replace("'ll", " will")
  x = re.sub(r"([0-9]+)000000", r"\1m", x)
  x = re.sub(r"([0-9]+)000", r"\1k", x)
  porter = PorterStemmer()
  pattern = re.compile('\W')
    
  if type(x) == type(''):
    x = re.sub(pattern, ' ', x)
    
    
  if type(x) == type(''):
    x = porter.stem(x)
    example1 = BeautifulSoup(x)
    x = example1.get_text()
               
    
  return x
    


* Function to Compute and get the features : With 2 parameters of Question 1 and Question 2

## 3.5 Advanced Feature Extraction (NLP and Fuzzy Features)

Definition:

* **Token**: You get a token by splitting sentence a space

* **Stop_Word** : stop words as per NLTK.

* **Word**  : A token that is not a stop_word

Features:

* **cwc_min** : Ratio of common_word_count to min lenghth of word count of Q1 and Q2 

* **cwc_min** = common_word_count / (min(len(q1_words), len(q2_words)) 

* **cwc_max** : Ratio of common_word_count to max lenghth of word count of Q1 and Q2 

* **cwc_max** = common_word_count / (max(len(q1_words), len(q2_words)) 

* **csc_min** : Ratio of common_stop_count to min lenghth of stop count of Q1 and Q2 

* **csc_min** = common_stop_count / (min(len(q1_stops), len(q2_stops)) 




* **csc_max** : Ratio of common_stop_count to max lenghth of stop count of Q1 and Q2

* **csc_max** = common_stop_count / (max(len(q1_stops), len(q2_stops)) 

* **ctc_min** : Ratio of common_token_count to min lenghth of token count of Q1 and Q2

* **ctc_min** = common_token_count / (min(len(q1_tokens), len(q2_tokens)) 


* **ctc_max** : Ratio of common_token_count to max lenghth of token count of Q1 and Q2

* **ctc_max** = common_token_count / (max(len(q1_tokens), len(q2_tokens)) 


* **last_word_eq**  : Check if First word of both questions is equal or not

* **last_word_eq** = int(q1_tokens[-1] == q2_tokens[-1]) 




* **first_word_eq** : Check if First word of both questions is equal or not

* **first_word_eq** = int(q1_tokens[0] == q2_tokens[0]) 


* **abs_len_diff** : Abs. length difference

* **abs_len_diff** = abs(len(q1_tokens) - len(q2_tokens)) 


* **mean_len** : Average Token Length of both Questions

* **mean_len** = (len(q1_tokens) + len(q2_tokens))/2 


* **fuzz_ratio** : https://github.com/seatgeek/fuzzywuzzy#usage http://chairnerd.seatgeek.com/fuzzywuzzy-fuzzy-string-matching-in-python/ 


* **fuzz_partial_ratio** : https://github.com/seatgeek/fuzzywuzzy#usage http://chairnerd.seatgeek.com/fuzzywuzzy-fuzzy-string-matching-in-python/ 


* **token_sort_ratio** : https://github.com/seatgeek/fuzzywuzzy#usage http://chairnerd.seatgeek.com/fuzzywuzzy-fuzzy-string-matching-in-python/ 

* **token_set_ratio** : https://github.com/seatgeek/fuzzywuzzy#usage http://chairnerd.seatgeek.com/fuzzywuzzy-fuzzy-string-matching-in-python/ 

* **longest_substr_ratio** : Ratio of length longest common substring to min lenghth of token count of Q1 and Q2
longest_substr_ratio = len(longest common substring) / (min(len(q1_tokens), len(q2_tokens))

In [ ]:
SAFE_DIV=0.0001
def get_tokens_feature(q1,q2):
  
  token_features=[0.0]*10
  
  #Token 
  q1_tokens=q1.split()
  q2_tokens=q2.split()
  
  #return if  
  if len(q1_tokens)==0 or len(q2_tokens)==0:
    return token_features
  
  #Words
  q1_words = set(word for word in q1_tokens if word not in Stopwords )
  q2_words = set(word for word in q2_tokens if word not in Stopwords )
  
  # Stop Words in Question
  q1_stopwords = set(word for word in q1_tokens if word in Stopwords )
  q2_stopwords = set(word for word in q2_tokens if word in Stopwords )
  
  # Counting common words in Question pair
  common_word_count =len(q1_words.intersection(q2_words))
  
  #Counting common Stopwords in Question pair
  common_stopwords_count =len(q1_stopwords.intersection(q2_stopwords))
  
  # Get the common words in Question pair
  common_token_count =len(set(q1_tokens).intersection(q2_tokens))
  
  #Common_Word_Count_Min
  token_features[0]= common_word_count/(min(len(q1_words),len(q2_words))+SAFE_DIV)
  
  #Common_Word_Count_Max
  token_features[1]= common_word_count/(max(len(q1_words),len(q2_words))+SAFE_DIV)
  
  #Common_StopWord_Count_Min
  token_features[2]= common_stopwords_count/(min(len(q1_stopwords),len(q2_stopwords))+SAFE_DIV)
  
  #Common_StopWord_Count_Max
  token_features[3]= common_stopwords_count/(max(len(q1_stopwords),len(q2_stopwords))+SAFE_DIV)
  
  #Common_StopWord_Count_Min
  token_features[4]= common_token_count/(min(len(q1_tokens),len(q2_tokens))+SAFE_DIV)
  
  #Common_StopWord_Count_Max
  token_features[5]= common_token_count/(max(len(q1_tokens),len(q2_tokens))+SAFE_DIV)
  
  # Last word of both question is same or not
  token_features[6]= int(q1_tokens[-1]==q2_tokens[-1])
  
  # First word of both question is same or not
  token_features[7]= int(q1_tokens[0]==q2_tokens[0])
  
  #Absolute_Token_length_difference 
  token_features[8]=abs(len(q1_tokens)-len(q2_tokens))
  
  #Average_Token_length_
  token_features[9]=(len(q1_tokens)+len(q2_tokens))/2
  
  return token_features


In [ ]:
#Longest Common substring using ditance library using lcsubstrings function
def get_longest_substr_ratio(a,b):
  substring=list(distance.lcsubstrings(a,b))
  if len(substring)==0:
    return 0
  else:
    return len(substring[0])/min(len(a),len(b)+1)

In [ ]:
def extract_features(df):
  print("Preprocessing...")
  #Preprocessing each question
  df["question1"]=df["question1"].fillna("").apply(Preprocessing)
  df["question2"]=df["question2"].fillna("").apply(Preprocessing)
  
  #Appending Advance Features
  token_features=df.apply(lambda x:get_tokens_feature(x["question1"],x["question2"]),axis=1 ) 
  
  #map(function_to_apply, list_of_inputs)
  df["cwc_min"]=       list(map(lambda x: x[0],token_features))
  df["cwc_max"]=       list(map(lambda x: x[1],token_features))
  df["csc_min"]=       list(map(lambda x: x[2],token_features))
  df["csc_max"]=       list(map(lambda x: x[3],token_features))
  df["ctc_min"]=       list(map(lambda x: x[4],token_features))
  df["ctc_max"]=       list(map(lambda x: x[5],token_features))
  df["last_word_ed"]=  list(map(lambda x: x[6],token_features))
  df["first_word_ed"]= list(map(lambda x: x[7],token_features))
  df["abs_len_diff"]=  list(map(lambda x: x[8],token_features))
  df["mean_len"]=      list(map(lambda x: x[9],token_features))

  print("fuzzy features..")
  df["token_set_ratio"]       = df.apply(lambda x: fuzz.token_set_ratio(x["question1"],x["question2"]),axis=1)
  df["token_sort_ratio"]      = df.apply(lambda x: fuzz.token_sort_ratio(x["question1"],x["question2"]),axis=1)
  df["longest_substr_ratio"]  = df.apply(lambda x:  get_longest_substr_ratio(x["question1"], x["question2"]), axis=1)
  df["fuzz_partial_ratio"]    = df.apply(lambda x: fuzz.partial_ratio(x["question1"], x["question2"]), axis=1)
  
  return df

In [ ]:
df = extract_features(data)

Preprocessing...
fuzzy features..


In [ ]:
df.head()

,id,qid1,qid2,question1,question2,is_duplicate,freq_qid1,freq_qid2,q1len,q2len,q1_n_words,q2_n_words,word_Common,word_Total,word_Share,freq_q1+q2,freq_q1-q2,cwc_min,cwc_max,csc_min,csc_max,ctc_min,ctc_max,last_word_ed,first_word_ed,abs_len_diff,mean_len,token_set_ratio,token_sort_ratio,longest_substr_ratio,fuzz_partial_ratio
0,0,1,2,what is the step by step guide to invest in sh...,what is the step by step guide to invest in sh...,0,1,1,66,57,14,12,10.0,23.0,0.434783,2,0,0.999980,0.833319,0.999983,0.999983,0.916659,0.785709,0.0,1.0,2.0,13.0,100,93,0.982759,100
1,1,3,4,what is the story of kohinoor koh i noor dia...,what would happen if the indian government sto...,0,4,1,51,88,8,13,4.0,20.0,0.200000,5,3,0.799984,0.399996,0.749981,0.599988,0.699993,0.466664,0.0,1.0,5.0,12.5,86,63,0.607843,75
2,2,5,6,how can i increase the speed of my internet co...,how can internet speed be increased by hacking...,0,1,1,73,59,14,10,4.0,24.0,0.166667,2,0,0.399992,0.333328,0.399992,0.249997,0.399996,0.285712,0.0,1.0,4.0,12.0,63,63,0.166667,47
3,3,7,8,why am i mentally very lonely how can i solve...,find the remainder when math 23 24 math i...,0,1,1,50,65,11,9,0.0,19.0,0.000000,2,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,2.0,12.0,28,24,0.040000,14
4,4,9,10,which one dissolve in water quikly sugar salt...,which fish would survive in salt water,0,3,1,76,39,13,7,2.0,20.0,0.100000,4,2,0.399992,0.199998,0.999950,0.666644,0.571420,0.307690,0.0,1.0,6.0,10.0,67,47,0.175000,56


**3.5.1 Analysis of extracted features**

**3.5.1.1 Plotting Word clouds**

* Creating Word Cloud of Duplicates and Non-Duplicates Question pairs

* We can observe the most frequent occuring words